# Construção de vocabulário

* Tokenizando textos em palavras e n-gramas (tokens)

* Como lidar com pontuação e emoticons fora do padrão de postagens nas redes sociais.

* Compactando vocabulários com stemming e lematização.

* Construidndo uma representação vetorial de uma instrução.

## Tokenizador

* Primeiro passo para um sistema de NLP é um bom vocabulário.

* Aqui iremos estudar algoritmos para separar uma string em palavras em pares, triplas, quadruplas e até quíntuplas. Estes pedaços são chamados de n-grams (n é igual ao tamanho dos pedaços).

* O uso de n-grams permite que sua máquina saiba sobre "ice cream", bem como os "ice" e "cream".

* No processamento de linguagem natural, compor um vetor numérico a partir de texto é um processo de extração de features "com perdas".

* No entanto, o bag-of-words (BOW), retêm o conteúdo de informações do texto suficiente para produzir modelos úteis de máquina.

* A tokenização é um tipo específico de segmentação de documento. A segmentação divide o texto em pedaços ou segmentos menores.

* A segmentação pode incluir a quebra de um documento em parágrafos, parágrafos em sentenças, frases em tokens (geralmente palavras) e pontuação.

* Em compiladores, um tokenizador usado para compilar linguagens de computador é chamado **lexer**.

* O vocubulário (o conjunto de todos os tokens válidos) para uma linguagem de computador é de lexico.

* Para um sistema de NLP, temos (equivalente aos compiladores):

    * Tokenizador: scanner, lexer, analisador lexical.

    * Vocabulário: léxico.

    * Analisador: compilador.

    * token ,termo ou n-gram: token, símbolo ou símbolo terminal.

* A tokenização é a primeira etapa de um pipeline de NLP.

* Objetivo: dividir dados não estruturados (texto em linguagem natural) em chunks de informações discretos.

* Essas contagens de ocorrência de tokens em um documento podem ser usadas diretamente como um vetor que representa este documento.

* Ideia: transformar uma sequeência não estruturada (documento textual) em uma esturtura de dados numérico adqueada para aprendizado de máquina.

* Codificação: A maneira mais simples de tokenizar uma frase é usar espaços em branco em uma string como o "delimitador" das palavras.

In [1]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
sentence.split()

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

In [2]:
import numpy as np
import pandas as pd

token_sequence = str.split(sentence)
vocab = sorted(set(token_sequence))

In [3]:
num_tokens = len(token_sequence)
vocab_size = len(vocab)

print(num_tokens, vocab_size)

10 10


In [4]:
onehot_vector = np.zeros((num_tokens, vocab_size), int)

for i, word in enumerate(token_sequence):
    onehot_vector[i, vocab.index(word)] = 1

In [5]:
pd.DataFrame(onehot_vector, columns=vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


Nesta representação do seu documento, cada linha é um vetor para uma única palavra.

Podemos usar o vetor $[0, 0, 0,	0,	1,	0,	0,	0,	0,	0]$ para representar a palavra "age" no pipeline.

Um "1" em uma coluna indica uma palavra do vocabulário que estava presente nessa posição no documento.

## Bag Of Words

Até aqui criamos uma representação com apneas um único documento (a frase utilizada).

Para ficar mais clara a ideia de bag-of-words precisamos trabalhar com vários documentos para termos um vocubalário (conjunto de palavras) significativamente maior.

In [6]:
sentences = "Thomas Jefferson began building Monticello at the age of 26.\n"
sentences += "Construction was done mstly by local masons and carpenters.\n"
sentences += "He moved into the South Pavillion in 1770.\n"
sentences += "Turning Monticello into a neoclassical masterpiece was Jefferson\n"

corpus = {}

for i, sent in enumerate(sentences.split('\n')):
   corpus[f'sent{i}'] = dict((tok, 1) for tok in sent.split())


df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.,...,moved,into,South,Pavillion,in,1770.,Turning,a,neoclassical,masterpiece
sent0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
sent1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0,...,1,1,1,1,1,1,0,0,0,0
sent3,0,1,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,1,1,1,1
sent4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Observe o resultado (várias colunas foram omitidas). Cada documento (sentença) possui apenas o valor "1" na posição do vocabulário em que há ocorrência daquela palavra.

* Agora iremos calcular essa sobreposição no pipeline para comparar documentos ou procurar documentos semelhantes, contando o número de tokens sobrepostos usando um produto escalar.

## Produto Escalar

* O produto escalar utiliza vetores de dimensões (tamanhos) iguais.

* Sua operação produz um único escalar como saída.

* O valor escalar gerado pelo produto escalar pode ser calculado multiplicando todos os elementos de um vetor por todos os elementos de um segundo vetor. 

In [7]:
v1 = np.array([1, 2, 3])
v2 = np.array([2, 3, 4])

np.dot(v1, v2)

20

###  Medindo o overlap entre bag-of-words

* Se pudermos medir a sobreposição de um conjunto de palavras para dois vetores, podemos obter uma boa estimativa de quão similares eles são.

* E esta é uma boa estimativa de quão similares elas são no significado.

In [8]:
df = df.T
print(df.sent0.dot(df.sent1))
print(df.sent0.dot(df.sent2))
print(df.sent0.dot(df.sent3))

0
1
2


* Veja que uma palavra foi usada em sent0 e sent2 (resposta 1).

* Da mesma forma, duas palavras do vocabulário foi usada em sent0 e sent3.

 * Essa sobreposição de palavras é uma medida de similaridade.

 * A frase sent1 foa a unica que não mencionou Jefferson ou Monticello diretamente, mas usou no conjunto de palvras completamente diferentes.

 * Este modelo visto aqui é uma forma de extração de features de frases.

 * Além de produtos escalares, outras operações de vetores pode ser usadas nestes casos: adição, subtração, OR, AND e assim por diante. Ou até mesmo distância euclidiana ou distância dos cossenos entre esses vetores.

### Melhorando a extração de tokens

* Agora podemos melhorar a extração de tokens separando melhor as palavras usando expressão regular (até então usamos apenas espaços em branco).

In [10]:
import re

pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)

print(tokens)

['Thomas', ' ', 'Jefferson', ' ', 'began', ' ', 'building', ' ', 'Monticello', ' ', 'at', ' ', 'the', ' ', 'age', ' ', 'of', ' ', '26', '.', '']


In [12]:
tokens = [x for x in tokens if x not in '- \t\n.,;!?']
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

In [18]:
from nltk.tokenize.casual import casual_tokenize

message = """RT  @TJMonticello Best day everrrrrrr at Monticello. Awesommmmmeeeee day  :*)"""

tokens = casual_tokenize(message)
print(tokens)

tokens = casual_tokenize(message, reduce_len=True, strip_handles=True)
print(tokens)

['RT', '@TJMonticello', 'Best', 'day', 'everrrrrrr', 'at', 'Monticello', '.', 'Awesommmmmeeeee', 'day', ':*)']
['RT', 'Best', 'day', 'everrr', 'at', 'Monticello', '.', 'Awesommmeee', 'day', ':*)']


## n-grams

* Um n-gram é uma sequência que contém até n elementos que foram extraídos de uma sequência. Em geral, os "elementos" de uma n-gram podem ser caractéres, sílabas, palavras ou até símbolos como "A", "T", "G", e "C" usandos para representar uma sequência de DNA.

* Aqui vamos focar nos n-gram de palavras (e não caractéres).

In [21]:
from nltk.util import ngrams

pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)

tokens = [x for x in tokens if x not in '- \t\n.,;!?']
print('Tokens: ', tokens)

tokens = list(ngrams(tokens, 2))
print('ngrams: ', tokens)

Tokens:  ['Thomas', 'Jefferson', 'began', 'building', 'Monticello', 'at', 'the', 'age', 'of', '26']
ngrams:  [('Thomas', 'Jefferson'), ('Jefferson', 'began'), ('began', 'building'), ('building', 'Monticello'), ('Monticello', 'at'), ('at', 'the'), ('the', 'age'), ('age', 'of'), ('of', '26')]


* O n-grams são obtidos na lista anterior como tuplas, mas podem ser unidos se você desejar que todos os tokens do pipeline sejam de caractéres.

* Isso permitirá que os estágios posteriores do pipeline esperem um tipo de dados consistente como entrada, sequência de strings:

In [22]:
tokens = pattern.split(sentence)

tokens = [x for x in tokens if x not in '- \t\n.,;!?']
two_grams = list(ngrams(tokens, 2))
token = [' '.join(x) for x in two_grams]
print(token)

['Thomas Jefferson', 'Jefferson began', 'began building', 'building Monticello', 'Monticello at', 'at the', 'the age', 'age of', 'of 26']


### Stop Words

* Stopwords são palavras comuns em qualquer idioma que ocorram com alta frequência, mas carregam muito menos informação substantiva sbre o significado de uma frase.

* Exemplos (lingua inglesa): a, na, the, this, and, or, of, on.

* Exemplos (lingua portuguesa): a, o, de, do, da, em, etc.

* POdemos filtrar arbitrariamente um conjunto de stopwords durante a tokenização.

* Aqui utilizamos alguns stopswords que serão ignorados ao percorrer a lista de tokens.

In [26]:
stop_words = ['a', 'an', 'the', 'on', 'off', 'this', 'is']
tokens = ['the', 'house', 'is', 'on', 'fire']

token_without_stopwords = [x for x in tokens if x not in stop_words]
print(token_without_stopwords)

['house', 'fire']


In [27]:
import nltk

nltk.download('stopwords')

stop_words = nltk.corpus.stopwords.words('english')
print(len(stop_words))

179
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/angelico/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
print(stop_words[0:20])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


In [29]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sklearn_stop_words

print(len(sklearn_stop_words))

318


In [39]:
sklearn_stop_words

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

### Normalização do vocabulário

* Técnicas de redução de vocabulário: normalizar o vocabulário para que tokens que significam algo semelhante sejam combinados em uma única forma normalizada.

* Isso reduz o número de tokens.

#### Case Folding

* A normalização de letras maiúsculas e minúsculas é uma maneira de reduzir o tamanho do seu vocabulário e generalizar o pipeline de NLP.

* Ajuda a consolidar palavras cujo objetivo é significar a mesma coisa (e que são escritas da mesma maneira) sob um único token.

In [40]:
tokens = ['House', 'Visitor', 'Center']
normalized_tokens = [x.lower() for x in tokens]
normalized_tokens

['house', 'visitor', 'center']

* Para um mecanismo de pesquisa sem normalização, se pesquisasse "Age", obteria um conjunto de documentos diferentes do que se pesquisasse "age".

* Ao normalizar o vocabulário no índice de pesquisa (assim como na consulta), garantimos que os dois tops de documentos sobre "age" sejam retornados, independentemente da capitalização na consulta do usuário.

### Stemming

* Outra técnica comun de normalização de vocabulário é eliminar as pequenas diferenças de significado.

- Ou seja, manter apenas a porção de uma palavra que resta após a remoção de prefixos e sufixos.

* A ideia é identificar um radical comun entre as várias formas de uma palavra (radicaização - stemming).

* Por exemplo, as palavras "housing" e "houses" compartilham a mesma porção, "house".

* O stemming remove sulfixos das palavras na tentativa de combinar palavras com significados semelhantes junto sob seu radical comum.

* Vamos a uma implementação simples de stemmer em Python puro que pode lidar com "s" à direita (língua inglesa).

In [45]:
def stem(phrase):
    return ' '.join([re.findall('^(.*ss|.*?)(s)?$', word)[0][0].strip("'") for word in phrase.lower().split()])

In [48]:
print(stem('houses'))
print(stem("Doctor house's call"))

house
doctor house call


* Dois dos algoritmos de stemming mais populares são o Porter e o Snowball.

* Esses algoritmos implementam regras mais complexas do que uma simples expressão reguar. Isso permite que o stemmer lide com as complexidades das regras de ortografia e final de palavras.

In [49]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
text = ' '.join([stemmer.stem(x).strip("'") for x in "dish washer's washed dishes".split()])

print(text)

dish washer wash dish


### Lemmatization

* Se tiver acesso a informações sobre conexões entre os significados de varias palavras, é possivel associar várias palavras, mesmo que a grafia seja bem diferente.

* Essa normalização que considera semântica de uma palavra - seu lema - é chamado lematização.

* Mais precisa do que técnicas anteriores (usa base de conhecimento de sinônimos).

* Alguns lematizadores usam a tag part of speech (POS) da palavra, além da ortografia, para melhorar a precisão.

* A tag POS para uma palavra indica seu papel na gramática de uma frase.

* Por exemplo, o POS substantivo é para palavras que se referem a "pessoas, lugares ou coisas", em uma frase. Um POS adjetivo é para uma palavra que modifique ou descreve substantivos.

* Um verbo se refere a uma ação.

* O pacote NLTK fornece funções para udentficar lemas de palavras.

* Observe que devemos informar a WordNetLemmatizer em qual POS você esta interessado (substantivo, adjetivo, etc).

* O lematizador NLTK usa um grafo de significados de palavras do Prineceton WordNet.

In [51]:
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lematizer = WordNetLemmatizer()

print(lematizer.lemmatize('better'))
print(lematizer.lemmatize('better', pos= 'a'))

[nltk_data] Downloading package wordnet to /home/angelico/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
better
good


In [52]:
print(lematizer.lemmatize('better'))
print(lematizer.lemmatize('better', pos='a'))
print(lematizer.lemmatize('good', pos='a'))
print(lematizer.lemmatize('goods', pos='a'))
print(lematizer.lemmatize('goods', pos='n'))
print(lematizer.lemmatize('goodness', pos='n'))
print(lematizer.lemmatize('best', pos='a'))

better
good
good
goods
good
goodness
best


In [53]:
WordNetLemmatizer?

Init signature: WordNetLemmatizer()
Docstring:     
WordNet Lemmatizer

Lemmatize using WordNet's built-in morphy function.
Returns the input word unchanged if it cannot be found in WordNet.

    >>> from nltk.stem import WordNetLemmatizer
    >>> wnl = WordNetLemmatizer()
    >>> print(wnl.lemmatize('dogs'))
    dog
    >>> print(wnl.lemmatize('churches'))
    church
    >>> print(wnl.lemmatize('aardwolves'))
    aardwolf
    >>> print(wnl.lemmatize('abaci'))
    abacus
    >>> print(wnl.lemmatize('hardrock'))
    hardrock
File:           ~/anaconda3/lib/python3.7/site-packages/nltk/stem/wordnet.py
Type:           type
Subclasses:     
